In [ ]:
import pandas as pd

### Plot all trajectories between two airports

In [ ]:
## Parameters
airport_orig = 'EGLL' # Heathrow
airport_dest = 'LGAV' # Athens
day='15-09-2023'

In [ ]:
## Read input data needed
## -- flights and points filed
## -- for flights transform times to datetime

# Flights
df_flights = pd.read_csv('./data/202309/Flights_20230901_20230930.csv.gz')
df_flights["FILED OFF BLOCK TIME"] = pd.to_datetime(df_flights["FILED OFF BLOCK TIME"], format="%d-%m-%Y %H:%M:%S")
df_flights["FILED ARRIVAL TIME"] = pd.to_datetime(df_flights["FILED ARRIVAL TIME"], format="%d-%m-%Y %H:%M:%S")
df_flights["ACTUAL OFF BLOCK TIME"] = pd.to_datetime(df_flights["ACTUAL ARRIVAL TIME"], format="%d-%m-%Y %H:%M:%S")
df_flights["ACTUAL ARRIVAL TIME"] = pd.to_datetime(df_flights["ACTUAL ARRIVAL TIME"], format="%d-%m-%Y %H:%M:%S")

# Flight Points Filed
df_points_filed = pd.read_csv('./data/202309/Flight_Points_Filed_20230901_20230930.csv.gz')

In [ ]:
## Filter flights between the two airports

# Filter airport
df_f = df_flights[(df_flights.ADEP==airport_orig) & (df_flights.ADES==airport_dest)].copy()

# Filter date arrival time
df_f = df_f[df_f['FILED ARRIVAL TIME'].apply(lambda x: x.date()==pd.to_datetime(day, format='%d-%m-%Y').date())].copy()


In [ ]:
# Print the number of flights arriving
print("Number flights between", airport_orig, "and", airport_dest, "are", len(df_f))

In [ ]:
## Filtering trajectory points

# Keep only trajectories arriving to the airport (use merge)
df_points_filed_f = df_points_filed.merge(df_f, on='ECTRL ID')

In [ ]:
## Plot trajectories between airports
from ectl_plots_utilities import plot_trajectory

f = None
a = None
for f in df_points_filed_f['ECTRL ID'].drop_duplicates():
    # Plot trajectories of flights filled
    f,a = plot_trajectory(df_points_filed_f[df_points_filed_f['ECTRL ID']==f], label=f, fig=f, ax=a)


### Arrivals at a given airport

In [ ]:
## Parameters
# Airport id
airport_id = 'EDDF'
# Day of interest
day='15-09-2023'
# Altitude cut trajectory
FL_max = 120

In [ ]:
## Read input data needed
## -- flights and points filed and actual
## -- for flights transform times to datetime

# Flights
df_flights = pd.read_csv('./data/202309/Flights_20230901_20230930.csv.gz')
df_flights["FILED OFF BLOCK TIME"] = pd.to_datetime(df_flights["FILED OFF BLOCK TIME"], format="%d-%m-%Y %H:%M:%S")
df_flights["FILED ARRIVAL TIME"] = pd.to_datetime(df_flights["FILED ARRIVAL TIME"], format="%d-%m-%Y %H:%M:%S")
df_flights["ACTUAL OFF BLOCK TIME"] = pd.to_datetime(df_flights["ACTUAL ARRIVAL TIME"], format="%d-%m-%Y %H:%M:%S")
df_flights["ACTUAL ARRIVAL TIME"] = pd.to_datetime(df_flights["ACTUAL ARRIVAL TIME"], format="%d-%m-%Y %H:%M:%S")

# Flight Points Filed
df_points_filed = pd.read_csv('./data/202309/Flight_Points_Filed_20230901_20230930.csv.gz')
# Flight Points Actual
df_points_actual = pd.read_csv('./data/202309/Flight_Points_Actual_20230901_20230930.csv.gz')

In [ ]:
## Filter flights going to a given airport on a given day

# Filter airport
df_f = df_flights[df_flights.ADES==airport_id].copy()

# Filter date arrival time
df_f = df_f[df_f['FILED ARRIVAL TIME'].apply(lambda x: x.date()==pd.to_datetime(day, format='%d-%m-%Y').date())].copy()


In [ ]:
# Print the number of flights arriving
print("Number flights arriving at", airport_id, "are: ", len(df_f))

In [ ]:
## Plot histogram of the distance of flights arriving to airport (actual)
df_f['Actual Distance Flown (nm)'].hist(bins=100)

In [ ]:
## Plot histogram of Arrival Time as planned per hour

# Extract the hour (0–23)
df_f["arrival_hour_filed"] = df_f["FILED ARRIVAL TIME"].dt.hour

# Plot histogram
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.hist(df_f["arrival_hour_filed"].dropna(), bins=24, range=(0, 24), edgecolor="black", color="skyblue")
plt.xticks(range(0, 25))
plt.xlabel("Filed Arrival Hour (UTC)")
plt.ylabel("Number of Flights")
plt.title("Distribution of Filed Arrival Times by Hour")
plt.grid(alpha=0.3)
plt.show()

In [ ]:
## Filtering trajectory points

# Keep only trajectories arriving to the airport (use merge)
df_points_filed_f = df_points_filed.merge(df_f, on='ECTRL ID')
df_points_actual_f = df_points_actual.merge(df_f, on='ECTRL ID')

# Keep only the segment of altitude lower than the selected flights
df_points_filed_f_filtered = df_points_filed_f[df_points_filed_f['Flight Level']<=FL_max]
df_points_actual_f_filtered = df_points_actual_f[df_points_actual_f['Flight Level']<=FL_max]

In [ ]:
## Keep only segments that are descending part of the flight

df_points_filed_f_filtered['Diff FL next'] = df_points_filed_f_filtered['Flight Level'].shift(-1) - df_points_filed_f_filtered['Flight Level']
df_points_actual_f_filtered['Diff FL next'] = df_points_actual_f_filtered['Flight Level'].shift(-1) - df_points_actual_f_filtered['Flight Level']

# Ensure the data is sorted correctly
df_points_actual_f_filtered = df_points_actual_f_filtered.sort_values(["ECTRL ID", "Sequence Number"])
df_points_filed_f_filtered = df_points_filed_f_filtered.sort_values(["ECTRL ID", "Sequence Number"])

def get_descent_phase(group):
    # Find the first index where the aircraft starts descending
    descent_start_idx = group.index[group["Diff FL next"] < 0]
    if len(descent_start_idx) == 0:
        # No descent detected → return empty
        return pd.DataFrame()
    # Take the first descent point index
    first_descent_idx = descent_start_idx[0]
    # Keep from that point onward
    return group.loc[first_descent_idx:]

# Apply to each flight
df_descent_only_actual = df_points_actual_f_filtered.groupby("ECTRL ID", group_keys=False).apply(get_descent_phase).reset_index(drop=True)
df_descent_only_filed = df_points_filed_f_filtered.groupby("ECTRL ID", group_keys=False).apply(get_descent_phase).reset_index(drop=True)



In [ ]:
## Plot trajectory at arrival planned and actual for flights at a given airport on a given day
from ectl_plots_utilities import plot_trajectory

f_id = 265355412

# Plot full trajectory of flight filled
plot_trajectory(df_points_filed_f[df_points_filed_f['ECTRL ID']==f_id], label='Full')

# Plot trajectory cut of flight filled
plot_trajectory(df_points_filed_f_filtered[df_points_filed_f_filtered['ECTRL ID']==f_id], label='Cut FL')

# Plot trajectory only descent
plot_trajectory(df_descent_only_filed[df_descent_only_filed['ECTRL ID']==f_id], label='Cut FL and descent')

In [ ]:
f_id = 265355412

# Plot filed and actual descent in same plot
f,a = plot_trajectory(df_descent_only_filed[df_descent_only_filed['ECTRL ID']==f_id], label='Filed')
plot_trajectory(df_descent_only_actual[df_descent_only_actual['ECTRL ID']==f_id], label='Actual', fig=f, ax=a, color='red')

In [ ]:
f_id = 265323937

# Plot filed and actual descent in same plot
f,a = plot_trajectory(df_descent_only_filed[df_descent_only_filed['ECTRL ID']==f_id], label='Filed')
plot_trajectory(df_descent_only_actual[df_descent_only_actual['ECTRL ID']==f_id], label='Actual', fig=f, ax=a, color='red')

In [ ]:
f_id = 265315393

# Plot filed and actual descent in same plot
f,a = plot_trajectory(df_descent_only_filed[df_descent_only_filed['ECTRL ID']==f_id], label='Filed')
plot_trajectory(df_descent_only_actual[df_descent_only_actual['ECTRL ID']==f_id], label='Actual', fig=f, ax=a, color='red')

### Demand at airspace AUA closer to a given airport

In [ ]:
airport_id = 'EGLL'
day='15-09-2023'

In [ ]:
## Read AUAs data
# Flight AUAs filed and actual
df_f_auas = pd.read_csv('./data/202309/Flight_AUAs_Filed_20230901_20230930.csv.gz')
df_a_auas = pd.read_csv('./data/202309/Flight_AUAs_Actual_20230901_20230930.csv.gz')

In [ ]:
# Filter flights by airport
df_f = df_flights[df_flights.ADES==airport_id].copy()

# Filter flights by date arrival time
df_f = df_f[df_f['FILED ARRIVAL TIME'].apply(lambda x: x.date()==pd.to_datetime(day, format='%d-%m-%Y').date())].copy()


In [ ]:
# Merge AUAs (filed and actual) with flights of that day and airport
df_f_auas_arrival = df_f_auas.merge(df_f, on='ECTRL ID')


In [ ]:
## Keep last AUA of filed filtered
# Drop duplicate combinations of (ECTRL ID, AUA ID) keeping only distinct AUAs per flight
df_unique = df_f_auas_arrival.drop_duplicates(subset=['ECTRL ID', 'AUA ID'])

# Get the last AUA ID for each ECTRL ID (based on row order)
last_auas = df_unique.groupby('ECTRL ID', as_index=False)['AUA ID'].last()

# Count how many times each AUA appears
last_auas.groupby(['AUA ID']).count()

In [ ]:
aua_interest = 'EGTTICTA'

In [ ]:
## Go back to all trajectories and filter entry and exit times filed and actual in AUA and day of interest

# Filter by aua interest
df_f_auas = df_f_auas[df_f_auas['AUA ID']== aua_interest]
df_a_auas = df_a_auas[df_a_auas['AUA ID']== aua_interest]

# Transform entry and exit time to datetime
df_f_auas['Entry Time']= pd.to_datetime(df_f_auas["Entry Time"], format="%d-%m-%Y %H:%M:%S")
df_a_auas['Entry Time']= pd.to_datetime(df_a_auas["Entry Time"], format="%d-%m-%Y %H:%M:%S")
df_f_auas['Exit Time']= pd.to_datetime(df_f_auas["Exit Time"], format="%d-%m-%Y %H:%M:%S")
df_a_auas['Exit Time']= pd.to_datetime(df_a_auas["Exit Time"], format="%d-%m-%Y %H:%M:%S")

# Filter by entry date = date of interest
df_f_auas = df_f_auas[df_f_auas['Entry Time'].apply(lambda x: x.date()==pd.to_datetime(day, format='%d-%m-%Y').date())]
df_a_auas = df_a_auas[df_a_auas['Entry Time'].apply(lambda x: x.date()==pd.to_datetime(day, format='%d-%m-%Y').date())]


In [ ]:
def flights_in_aua(df, freq="15min"):
    # Ensure datetime type
    df = df.copy()
    df["Entry Time"] = pd.to_datetime(df["Entry Time"])
    df["Exit Time"] = pd.to_datetime(df["Exit Time"])

    # Floor to the nearest hour for start, ceil for end
    start_time = df["Entry Time"].min().floor("h")
    end_time = df["Exit Time"].max().ceil("h")

    # Build time range using the chosen frequency
    time_range = pd.date_range(start=start_time, end=end_time, freq=freq)

    results = []
    for start in time_range:
        end = start + pd.to_timedelta(freq)
        inside = ((df["Entry Time"] < end) & (df["Exit Time"] > start)).sum()
        results.append({"start": start, "end": end, "n_flights": inside})

    return pd.DataFrame(results)


f_inside_f_15 = flights_in_aua(df_f_auas, freq="15min")
f_inside_a_15 = flights_in_aua(df_a_auas, freq="15min")
f_inside_f_30 = flights_in_aua(df_f_auas, freq="30min")
f_inside_a_30 = flights_in_aua(df_a_auas, freq="30min")
f_inside_f_60 = flights_in_aua(df_f_auas, freq="60min")
f_inside_a_60 = flights_in_aua(df_a_auas, freq="60min")


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.step(f_inside_f_15["start"], f_inside_f_15["n_flights"], where="post", label="15 min f")
plt.step(f_inside_a_15["start"], f_inside_a_15["n_flights"], where="post", label="15 min a")
plt.step(f_inside_f_30["start"], f_inside_f_30["n_flights"], where="post", label="30 min f")
plt.step(f_inside_a_30["start"], f_inside_a_30["n_flights"], where="post", label="30 min a")
plt.step(f_inside_f_60["start"], f_inside_f_60["n_flights"], where="post", label="60 min f")
plt.step(f_inside_a_60["start"], f_inside_a_60["n_flights"], where="post", label="60 min a")
plt.legend()
plt.xlabel("Time")
plt.ylabel("Flights inside AUA")
plt.title("Number of flights inside AUA over time")
plt.show()
